# Frequently Asked Questions:

### It is recommended to take a look at the `BasicUsage` notebook before looking at this

In [1]:
import tequila as tq
import numpy

# Can I avoid re-translation/compilation on my objectives/circuits?

Yes you can. By calling `tq.compile` instead of `tq.simulate`.
This will give you back a callable objective.  
Check also the `basic usage` tutorial notebook

In [2]:
U = tq.gates.H(target=1) + tq.gates.Rx(angle="a", target=0, control=1)

# simulate the wavefunction with different variables
wfn0 = tq.simulate(U, variables={"a": 1.0})
wfn1 = tq.simulate(U, variables={"a": 2.0})

print(wfn0)
print(wfn1)

# the same, but avoiding re-compilation
# Note that your compiled object is translated to a quantum backend
# if the backend was not set, tequila it will pick the default which depends
# on which backends you have installed. You will seee it in the printout of the
# compiled circuits
compiled_U = tq.compile(U)
wfn0 = compiled_U(variables={"a":1.0})
wfn1 = compiled_U(variables={"a":2.0})

print("compiled circuit:", compiled_U)
print(wfn0)
print(wfn1)


# With Objectives it works in the same way
H = tq.paulis.Y(0)
E = tq.ExpectationValue(H=H, U=U)
objective = E**2 + 1.0

# simulate the objective with different variables
result0 = tq.simulate(objective, variables={"a": 1.0})
result1 = tq.simulate(objective, variables={"a": 2.0})

print("compiled objective:", objective)
print(result0)
print(result1)

# compile and then simulate
compiled_objective = tq.compile(objective)
result0 = compiled_objective(variables={"a":1.0})
result1 = compiled_objective(variables={"a":2.0})

print("compiled objective:", compiled_objective)
print(result0)
print(result1)

+0.7071|00> +0.6205|01> -0.3390i|11> 
+0.7071|00> +0.3821|01> -0.5950i|11> 
compiled circuit: <tequila.simulators.simulator_qulacs.BackendCircuitQulacs object at 0x7fdb4360d550>
+0.7071|00> +0.6205|01> -0.3390i|11> 
+0.7071|00> +0.3821|01> -0.5950i|11> 
compiled objective: Objective with 1 unique expectation values
variables = [a]
types     = not compiled
1.1770184
1.2067055
compiled objective: Objective with 1 unique expectation values
variables = [a]
types     = [<class 'tequila.simulators.simulator_qulacs.BackendExpectationValueQulacs'>]
1.1770184
1.2067055


# how can I run on a real quantum computer?
IBM's Qiskit can be used to run on some of IBM's public accessible quantum computers.
All you need for this is an ibm account (Follow the instructions under "Configure your IBM Quantum Experience credentials" here: https://github.com/Qiskit/qiskit-ibmq-provider).  
Tequila also supports Rigetti's PyQuil and Google's Cirq, but currently there are no publicly available devices.

Here is a small  example with Qiskit (you need to have qiskit installed, and an activaged IBMQ account for this):  
If you are already familiar with Qiskit and have special access rights you can also specific the provider by passing the keyword   
`qiskit_provider = <initialized_provider_instance>`   
Otherwise the default public provider will be initialized.  
Alternatively you can also externally initialize your backend and pass this down instead of a string

You always need to set samples if you intend to run on a real backend

I if you have access to Rigettis hardware you can use `backend="pyquil"` and pass down  
`pyquil_backend = dictonary`  
where the dictionary contains the instruction which go into pyquils `get_qc` function.

In [10]:
U = tq.gates.Ry(angle="a", target=0)
H = tq.paulis.X(0)
E = tq.ExpectationValue(H=H, U=U)

# simulate the square of the expectation value with a specific set of variables
result = tq.simulate(E**2, variables={"a":1.0}, samples=1000, backend="qiskit")

# run the same thing on one of IBM's quantum computers (check your ibm account for more information and keywords)
# note that the names of the computer might have changed
# here we use the 1-Qubit 'ibmq_armonk' 
result = tq.simulate(E**2, variables={"a":1.0}, samples=1000, backend="qiskit", qiskit_backend="ibmq_armonk") 

# In the SciPy Optimizers you can pass down all qiskit related options with the 
# `backend_options` key
# again you can also pass down a initialized qiskit backend object
backend_options = {"qiskit_backend":"ibmq_armonk"}
result = tq.optimizer_scipy.minimize(E**2, initial_values={"a":1.0}, samples=1000, backend="qiskit", backend_options=backend_options)

ObjectiveType is <class 'tequila.objective.objective.Objective'>
Starting BFGS optimization
Objective: 1 expectationvalues
Gradients: 3 expectationvalues (min=3, max=3)
Hessian: None

backend: <class 'tequila.simulators.simulator_qiskit.BackendExpectationValueQiskit'>
samples: 1000
1 active variables
E= 0.4489  angles= {a: 1.0}  samples= 1000
E= 0.11022399  angles= {a: 0.2940879464149475}  samples= 1000
E= 0.527076  angles= {a: -0.9714051337239711}  samples= 1000
E= 0.009604  angles= {a: 0.014596779994017095}  samples= 1000
E= 0.029584002  angles= {a: -0.23439378421529336}  samples= 1000
E= 0.0014439999  angles= {a: -0.06194057825106759}  samples= 1000
E= 0.00019600001  angles= {a: -0.10840037285555679}  samples= 1000
E= 0.024964003  angles= {a: -0.0830396865576113}  samples= 1000
E= 0.0081  angles= {a: -0.10793984869970413}  samples= 1000
E= 0.0025000002  angles= {a: -0.10839985851899529}  samples= 1000
E= 0.0017639999  angles= {a: -0.10840037285334388}  samples= 1000
E= 0.0067240004 

# Can I compile Objectives into different backends?
Yes you can. Tequila will print a warning if this happens. Warnings can be ignored by filtering them out (see the python warnings documentation)  

If a compiled circuit is used as input to compile then tequila will re-compile the circuit to the new backend (it it differs from the previous one)  

If a compiled objective is used as input to compile then tequila will only compile non-compiled expectationvalues into the different backend. Already compiled expectation values will remain untouched  

Note that you need at least two different backends for the following cell to execute.  
Just change the key to whatever you have installed.

In [3]:
backend1 = "qulacs"
backend2 = "cirq"

U = tq.gates.X(target=[0,1])
print("Example Circuit: ", U)
compiled_1 = tq.compile(U, backend=backend1)
compiled_2 = tq.compile(compiled_1, backend=backend2)
print("Circuit compiled to {} -> ".format(backend1), compiled_1)
print("Circuit compiled to {} -> ".format(backend1), compiled_1)

H = tq.paulis.X(0)*tq.paulis.Y(1) + tq.paulis.X([0,1])
print("\nmake objective with H = ", H)
objective = tq.ExpectationValue(H=H, U=U)
compiled_1 = tq.compile(objective, backend=backend1)

print("\nExpectationValues of objective 1:")
print(compiled_1)
    
objective2 = compiled_1 + objective # Its recommended to avoid those hybrids, but in principle it works

print("\nExpectationValues of partly compiled objective:")
print(objective2)
    
compiled_2 = tq.compile(objective2, backend=backend2)
print("\nExpectationValues of hybdrid compiled objective:")
print(compiled_2)




Example Circuit:  circuit: 
X(target=(0, 1), control=())

Circuit compiled to qulacs ->  <tequila.simulators.simulator_qulacs.BackendCircuitQulacs object at 0x7fdad8377898>
Circuit compiled to qulacs ->  <tequila.simulators.simulator_qulacs.BackendCircuitQulacs object at 0x7fdad8377898>

make objective with H =  +1.0000X(0)Y(1)+1.0000X(0)X(1)

ExpectationValues of objective 1:
Objective with 1 unique expectation values
variables = []
types     = [<class 'tequila.simulators.simulator_qulacs.BackendExpectationValueQulacs'>]

ExpectationValues of partly compiled objective:
Objective with 2 unique expectation values
variables = []
types     = partially compiled to [<class 'tequila.simulators.simulator_qulacs.BackendExpectationValueQulacs'>]

ExpectationValues of hybdrid compiled objective:
Objective with 2 unique expectation values
variables = []
types     = [<class 'tequila.simulators.simulator_cirq.BackendExpectationValueCirq'>, <class 'tequila.simulators.simulator_qulacs.BackendExpectat

 /home/jsk/projects/OpenVQE/src/tequila/simulators/simulator_api.py:243: TequilaWarning:Looks like the circuit was already compiled for another backend.
Changing from <class 'tequila.circuit.circuit.QCircuit'> to <class 'tequila.simulators.simulator_cirq.BackendCircuitCirq'>

 /home/jsk/projects/OpenVQE/src/tequila/simulators/simulator_api.py:190: TequilaWarning:Looks like part the objective was already compiled for another backend.
Found ExpectationValue of type <class 'tequila.simulators.simulator_qulacs.BackendExpectationValueQulacs'> and <class 'tequila.simulators.simulator_cirq.BackendExpectationValueCirq'>
... proceeding with hybrid



# How do I transform Measurements into Hamiltonians?

We can not answer this question in general, but we can try to give a small example here.  

Assume you have a quantum circuit with $4$ Qubits and you are measuring Qubit $0$ and $2$. 
You define your cost function in the following way:

$$
L(AB) = A + B, \qquad A,B \in \left\{ 0,1 \right\}  
$$

meaning you accumulate the number of ones measured in your circuit.  

The corresponding expectationvalue would be  

$$
L = \langle \Psi \rvert H \lvert \Psi \rangle \qquad H = 1 - \frac{1}{2}\left(Z(0) + Z(1)\right) 
$$

The Hamiltonian could also be written as

$$
H = 2\lvert 11 \rangle \langle 11 \rvert + \lvert 10 \rangle \langle 10 \rvert + \lvert 01 \rangle \langle 01 \rvert
$$

Tequila provides the convenience function `tq.gates.Projector` to initialize Hamiltonians like that


In [4]:
2*tq.paulis.Projector("|11>") + tq.paulis.Projector("|01>") + tq.paulis.Projector("|10>")

+1.0000-0.5000Z(1)-0.5000Z(0)

The projector can also be initialized with more structured `QubitWaveFunction`s which can itself be initialized from array or string.  
Here are some examples

In [5]:
wfn = tq.QubitWaveFunction.from_string("1.0*|00> + 1.0*|11>")
wfnx = tq.QubitWaveFunction.from_array(arr=[1.0, 0.0, 0.0, 1.0])
print(wfn == wfnx)
wfn = wfn.normalize()
print(wfn)

P = tq.paulis.Projector(wfn=wfn)
print("P = ", P)

True
+0.7071|00> +0.7071|11> 
P =  +0.2500+0.2500Z(0)Z(1)+0.2500X(0)X(1)-0.2500Y(0)Y(1)


Apart from `Projector` there is also `KetBra` which intialized more general operators like
$$
\lvert \Psi \rangle \langle \Phi \rvert
$$

Keep in mind that those are not hermitian.  
But they can be split up into their hermitian and anti hermitian part where both can then be used as hamiltonians for expectationvalues.

If the `hermitian = True` key is set, the function returns the hermitian version of the operator (which is the same as the hermitian part of the old operator)

$$
\frac{1}{2}\left(\lvert \Psi \rangle \langle \Phi \rvert + \lvert \Phi \rangle \langle \Psi \rvert \right)
$$

In [6]:
wfn1 = tq.QubitWaveFunction.from_string("1.0*|00> + 1.0*|11>").normalize()

op = tq.paulis.KetBra(bra=wfn1, ket="|00>")

H1, H2 = op.split()

print("operator=", op)
print("hermitian part      = ", H1)
print("anti-hermitian part =", H2)

H = tq.paulis.KetBra(bra=wfn1, ket="|00>", hermitian=True)
print("hermitianized operator = ", H)


operator= +0.1768+0.1768Z(1)+0.1768Z(0)+0.1768Z(0)Z(1)+0.1768X(0)X(1)+0.1768iX(0)Y(1)+0.1768iY(0)X(1)-0.1768Y(0)Y(1)
hermitian part      =  +0.1768+0.1768Z(1)+0.1768Z(0)+0.1768Z(0)Z(1)+0.1768X(0)X(1)-0.1768Y(0)Y(1)
anti-hermitian part = +0.1768iX(0)Y(1)+0.1768iY(0)X(1)
hermitianized operator =  +0.1768+0.1768Z(1)+0.1768Z(0)+0.1768Z(0)Z(1)+0.1768X(0)X(1)-0.1768Y(0)Y(1)


# Can I do basic operations on wavefunctions and operators without quantum backends?

In principle yes. But keep in mind that tequila was not made for this.  
However, some of those operations might come in handy for debugging or small examples.  

You can not execute circuits without a simulator since they are just abstract data types (no matrices or anything). Tequila has however its own small debug simulator `backend = symbolic` but there is no reason to use it if you have any other quantum backend installed.

Hamiltonians can be converted to matrices.

We give a few examples here

In [7]:
wfn = tq.QubitWaveFunction.from_string("1.0*|0> + 1.0*|1>").normalize()
H = 1.0/numpy.sqrt(2.0)*(tq.paulis.Z(0) + tq.paulis.X(0))
wfn2 = wfn.apply_qubitoperator(H).simplify()

print("|wfn>  = ", wfn)
print("H      = ", H)
print("H|wfn> = ", wfn2)

|wfn>  =  +0.7071|0> +0.7071|1> 
H      =  +0.7071Z(0)+0.7071X(0)
H|wfn> =  +1.0000|0> 


In [8]:
wfn1 = tq.QubitWaveFunction.from_string("1.0*|0> + 1.0*|1>").normalize()
wfn2 = tq.QubitWaveFunction.from_string("1.0*|0> - 1.0*|1>").normalize()
print("<wfn1|wfn2> = ", wfn1.inner(wfn2))

<wfn1|wfn2> =  0j


In [9]:
H = 1.0/numpy.sqrt(2.0)*(tq.paulis.Z(0) + tq.paulis.X(0))
print(H.to_matrix())

[[ 0.70710678+0.j  0.70710678+0.j]
 [ 0.70710678+0.j -0.70710678+0.j]]


# Can I import an Hamiltonian from OpenFermion?

Yes! OpernFermion is currently tequilas backend for Hamiltonians, which makes importing from it straight forward.
You just need to wrap the OpenFermion QubitOperator into tequilas QubitHamiltonian.

We show a few examples

In [11]:
from openfermion import QubitOperator

# get OpenFermion QubitOperator from tequila QubitHamiltonian
H = tq.paulis.X(0)
of_operator = H.to_openfermion()

print("{} = {}".format(type(H), H))
print("{} = {}".format(type(of_operator), of_operator))

# init tequila QubitHamiltonian with OpenFermion QubitOperator
H = tq.QubitHamiltonian.from_openfermion(of_operator)
print("{} = {}".format(type(H), H))

# initialization from file os often read in the string form
of_string = str(of_operator)
tq_string = str(H)

print(of_string)
print(tq_string)

H = tq.QubitHamiltonian.from_string(string=of_string, openfermion_format=True)
print(H)
H = tq.QubitHamiltonian.from_string(string=tq_string, openfermion_format=False)
print(H)

<class 'tequila.hamiltonian.qubit_hamiltonian.QubitHamiltonian'> = +1.0000X(0)
<class 'openfermion.ops._qubit_operator.QubitOperator'> = 1.0 [X0]
<class 'tequila.hamiltonian.qubit_hamiltonian.QubitHamiltonian'> = +1.0000X(0)
1.0 [X0]
+1.0000X(0)
+1.0000X(0)
+1.0000X(0)


# Can I compile into a regular function instead of one which takes dictionaries?

Not recommended but yes. The order of the function arguments is the order you get from `extract_variables`

In [13]:
U = tq.gates.Ry(angle="a", target=0)
U += tq.gates.X(power = "b", target=1)
H = tq.QubitHamiltonian.from_string("X(0)Z(1)")
E = tq.ExpectationValue(H=H, U=U)

f = tq.compile_to_function(E)

print("order is : ", E.extract_variables())
print(f(0.5, 1.0))
print(tq.simulate(E, variables={"a":0.5, "b":1.0}))


order is :  [a, b]
-0.47942553860420106
-0.47942553860420106


If you also want to fix the samples and other entries to your compiled objective you can build wrappers

In [14]:
def mywrapper(compiled_obj, samples):
    return lambda *x: compiled_obj(*x, samples=samples)

wrapped = mywrapper(f, samples=100)

# don't expect same results, since samples are taken individually
print(wrapped(1.0, 0.5)) # always takes 100 samples
print(f(1.0, 0.5, samples=100)) # samples need to be given
print(f(1.0, 0.5, samples=1000)) # but sampling rate can be changed
print(f(1.0, 0.5)) # you can go back to full simulation which you cannot with the wrapped function

-0.16
0.18
-0.032
-3.6781866386004225e-08
